# Metrics extraction

In [9]:
import pandas as pd
import numpy as np

In [10]:
class DilithiumStats:
    def __init__(self, design_id: str, initial_tv: int, last_tv: int):
        self.base_template_vars = {
            "design_id": design_id,
            "initial_tv": initial_tv,
            "last_tv": last_tv
        }
        self.file_template = "old/{op_type}_{design_id}_lvl{sec_level}_tv{initial_tv}_{last_tv}.csv"

    def get_filename(self, op_type, sec_level):
        return self.file_template.format(op_type=op_type, sec_level=sec_level, **self.base_template_vars)

    def get_keygen_data(self):
        keygen_lvl2_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="2")).drop(labels="success", axis=1)
        keygen_lvl3_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="3")).drop(labels="success", axis=1)
        keygen_lvl5_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="5")).drop(labels="success", axis=1)
        keygen_lvl5_df = keygen_lvl5_df.rename(columns={col: f"{col}_lvl5" for col in keygen_lvl5_df.columns if "cycles" in col})
        keygen_df = pd.merge(keygen_lvl2_df, keygen_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        keygen_df = pd.merge(keygen_df, keygen_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return keygen_df
    
    def get_sign_data(self):
        sign_lvl2_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="2")).drop(labels="success", axis=1)
        sign_lvl3_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="3")).drop(labels="success", axis=1)
        sign_lvl5_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="5")).drop(labels="success", axis=1)

        for current_sign_df in [sign_lvl2_df, sign_lvl3_df, sign_lvl5_df]:
            current_sign_df['rejects_tries_count'] = (current_sign_df['rejects_count'] + 1)
            current_sign_df.drop(labels="rejects_count", axis=1, inplace=True)

        sign_df = pd.merge(sign_lvl2_df, sign_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        sign_df = pd.merge(sign_df, sign_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return sign_df
    
    def get_verify_data(self):
        verify_lvl2_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="2")).drop(labels="success", axis=1)
        verify_lvl3_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="3")).drop(labels="success", axis=1)
        verify_lvl5_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="5")).drop(labels="success", axis=1)
        verify_df = pd.merge(verify_lvl2_df, verify_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        verify_df = pd.merge(verify_df, verify_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return verify_df

In [11]:
lr_stats = DilithiumStats(design_id="perf0", initial_tv=0, last_tv=9)
hp_stats = DilithiumStats(design_id="perf1", initial_tv=0, last_tv=9)

## Key generation

In [12]:
hp_keygen_df = hp_stats.get_keygen_data()
lr_keygen_df = lr_stats.get_keygen_data()

In [13]:
hp_keygen_df.describe()

,test_num,total_cycles_lvl2,total_cycles_lvl3,total_cycles_lvl5
count,10.00000,10.000000,10.000000,10.000000
mean,4.50000,4605.300000,7898.300000,13511.800000
std,3.02765,3.267687,2.110819,5.884065
min,0.00000,4600.000000,7895.000000,13503.000000
25%,2.25000,4603.000000,7897.000000,13509.000000
50%,4.50000,4605.500000,7898.500000,13509.000000
75%,6.75000,4607.750000,7899.000000,13515.250000
max,9.00000,4610.000000,7903.000000,13521.000000


In [14]:
lr_keygen_df.describe()

,test_num,load_cycles_lvl2,exec_cycles_lvl2,unload_cycles_lvl2,total_cycles_lvl2,load_cycles_lvl3,exec_cycles_lvl3,unload_cycles_lvl3,total_cycles_lvl3,load_cycles_lvl5,exec_cycles_lvl5,unload_cycles_lvl5,total_cycles_lvl5
count,10.00000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000
mean,4.50000,10.0,18724.600000,2882.0,21616.600000,10.0,33045.300000,5586.0,38641.300000,10.0,50987.600000,5602.0,56599.600000
std,3.02765,0.0,56.042048,0.0,56.042048,0.0,59.355895,0.0,59.355895,0.0,94.023874,0.0,94.023874
min,0.00000,10.0,18646.000000,2882.0,21538.000000,10.0,32966.000000,5586.0,38562.000000,10.0,50837.000000,5602.0,56449.000000
25%,2.25000,10.0,18682.000000,2882.0,21574.000000,10.0,32986.750000,5586.0,38582.750000,10.0,50918.000000,5602.0,56530.000000
50%,4.50000,10.0,18721.000000,2882.0,21613.000000,10.0,33061.500000,5586.0,38657.500000,10.0,50993.500000,5602.0,56605.500000
75%,6.75000,10.0,18751.000000,2882.0,21643.000000,10.0,33090.250000,5586.0,38686.250000,10.0,51030.000000,5602.0,56642.000000
max,9.00000,10.0,18813.000000,2882.0,21705.000000,10.0,33132.000000,5586.0,38728.000000,10.0,51163.000000,5602.0,56775.000000


# Signing

In [15]:
hp_sign_df = hp_stats.get_sign_data()

In [16]:
hp_sign_df

,test_num,total_cycles_lvl2,rejects_tries_count_lvl2,total_cycles_lvl3,rejects_tries_count_lvl3,total_cycles,rejects_tries_count
0,0,10942,1,24290,2,268200,23
1,1,22578,3,16175,1,24843,1
2,2,22581,3,32397,3,24857,1
3,3,16786,2,113479,13,135477,11
4,4,22606,3,40500,4,46979,3
5,5,10965,1,121591,14,91236,7
6,6,51719,8,16178,1,46990,3
7,7,80844,13,16174,1,35918,2
8,8,40087,6,105382,12,113316,9
9,9,10984,1,24291,2,146512,12


## Reject loop time estimation

In [17]:
def estimate_reject_loop_time(dataframe, cycles_col_name: str):
    result_list = []

    for sec_lvl in [2, 3, 5]:
        estimatives = []
        tries_col_name_lvl = f'rejects_tries_count_lvl{sec_lvl}'
        cycles_col_name_lvl = f'{cycles_col_name}_lvl{sec_lvl}'

        retries_num_list = sorted(list(dataframe[tries_col_name_lvl].unique()))
        for idx in range(1, len(retries_num_list)):
            current_retry_num = retries_num_list[idx]
            previous_retry_num = retries_num_list[idx-1]
            retry_num_delta = current_retry_num - previous_retry_num

            current_time = dataframe[dataframe[tries_col_name_lvl] == current_retry_num][cycles_col_name_lvl].mean()
            previous_time = dataframe[dataframe[tries_col_name_lvl] == previous_retry_num][cycles_col_name_lvl].mean()
            estimatives.append((current_time-previous_time)/retry_num_delta)

        result_list.append(np.mean(estimatives))

    return tuple(result_list)

In [ ]:
estimate_reject_loop_time(dataframe=hp_sign_df, cycles_col_name="total_cycles")